In [ ]:
import matplotlib.pyplot as plt
import gym
import numpy as np
from ppo_torch_conv1d import Agent
from utils import plot_learning_curve
from utils import plot_test_curve
import uniform_gust_test_environment_diff as u_gust_env

import time


if __name__ == '__main__':
    steps_ep = 200
    seed = 0  #ahave done(0 - 4)
    speed = 15
    batch_size = 5
    n_epochs = 4
    alpha = 0.00003
    N = 20
    UvS = 'uniform'
    defls = [600,900,1200]
    defl = defls[0]
    tap = 6
    end = True
        
    if end:
        saved = 'end'
    else:
        saved = 'top'
  
   
    env = u_gust_env.mfc_gust_Env(deflect = defl, steps_per_ep = steps_ep, seed=seed)
    uvs = 'uni'
    obs_hist_shape = (tap+1,steps_ep)
    act_avg = True
    act_mode = False
    
    
    figure_file = 'plots/wing_test_seed0.png'    
    # File Directory for saving test data (note that data was saved in a different format than presented here)
    file = 'Data/high_lift/speed'+str(speed)+'/uniform_comp/seed'+str(seed)+'/'+str(tap)+'tap'
    # Load trained agent file to test
    agent_file = 'Mar_31_'+uvs+'_sp' +str(speed)+'_'+str(tap)+'tap_seed'+str(seed)+'/'
    agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
                    alpha=alpha, n_epochs=n_epochs, 
                    input_dims=env.observation_space.shape, 
                    fc1_dims=512, fc2_dims=512, chk_dir=file+'/ppo/'+agent_file,
                    seed=seed)
  

    
    agent.load_models(end=end)
    best_score = env.reward_range[0]
    
    for defln in defls:
        
        n_games = 20
        test_num = 0
        for i in range(n_games):
            env.end()
            if i%2==0:
                defl = defln
                test_num += 1
            else:
                defl = -defln
            print('deflection: '+str(defl))
            
            if defl>0:
                direct = 'up'
            else:
                direct = 'down'
            
          
            env = u_gust_env.mfc_gust_Env(deflect = defl, steps_per_ep = steps_ep, seed=seed)
                
            observation = env.reset()
            done = False
            score = 0
            learn_iters = 0
            avg_score = 0
            n_steps = 0
        
            lift_history = []
            drag_history = []
            time_hist = [0]
            obs_hist = np.zeros(obs_hist_shape)
            act_hist = []
            t = time.time()
            while not done:
                
                if not env.g_rotating:
                    
                    action, prob, val = agent.choose_action(observation, avg=act_avg, mode=act_mode)
                    
                    observation_, reward, done, lift, drag = env.step(action)
                    score += reward
                    agent.remember(observation, action, prob, val, reward, done)
                    observation = observation_
                    obs_hist[:,n_steps] = observation[:,-1]
                    lift_history.append(lift)
                    drag_history.append(drag)
                    t_ = time.time()
                    dt = t_ - t
                    t = t_
                    time_hist.append(dt+time_hist[-1])
                    n_steps+=1
                else:
                    env.check_table()
                    env.pause_for_timing()
    
                x = [i+1 for i in range(len(lift_history))]
        
        
            np.savetxt(file+'/testing/'+str(tap)+'tap_Test_'+uvs+'_sp'+str(speed)+'_seed'+str(seed)+
                       '_'+saved+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'.csv', (lift_history,drag_history), delimiter =",")
            np.savetxt(file+'/testing/'+str(tap)+'tap_Test_'+uvs+'_sp'+str(speed)+'_seed'+str(seed)+
                       '_'+saved+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'_obs.csv', obs_hist, delimiter=',')
            np.savetxt(file+'/testing/'+str(tap)+'tap_test_'+uvs+'_sp'+str(speed)+'_seed'+str(seed)+
                       '_'+saved+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'_time.csv', time_hist, delimiter=',')
            #np.savetxt(UvS+'/speed'+str(speed)+'/Baseline/baseline_sp'+str(speed)+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'.csv', (lift_history,drag_history), delimiter =",")
            #np.savetxt(UvS+'/speed'+str(speed)+'/Baseline/baseline_sp'+str(speed)+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'_obs.csv', obs_hist, delimiter=',')
            #np.savetxt(UvS+'/speed'+str(speed)+'/Baseline/baseline_sp'+str(speed)+'_g'+str(defl)+'_'+direct+'_'+str(test_num)+'_timing.csv', time_hist, delimiter=',')
            if i == 0:
                plot_test_curve(x,lift_history, drag_history, figure_file, avg_of=10)
            elif i == 1:
                plot_test_curve(x,lift_history, drag_history, figure_file, avg_of=10)
    env.end()
  